In [6]:
import process_prediction.config as config
import process_prediction.predictor as test
from process_prediction.preprocessor import Preprocessor
import process_prediction.utils as utils

from IPython.display import display, HTML
from process_prediction.explanation.LSTM.LSTM_bidi import *
from process_prediction.explanation.util.heatmap import html_heatmap


args = config.load()
preprocessor = Preprocessor(args)
output = utils.load_output()
utils.clear_measurement_file(args)


usage: ipykernel_launcher.py [-h] [--explain EXPLAIN]
                             [--dnn_num_epochs DNN_NUM_EPOCHS]
                             [--dnn_architecture DNN_ARCHITECTURE]
                             [--task TASK] [--learning_rate LEARNING_RATE]
                             [--embedding_dim EMBEDDING_DIM]
                             [--embedding_epochs EMBEDDING_EPOCHS]
                             [--num_folds NUM_FOLDS]
                             [--cross_validation CROSS_VALIDATION]
                             [--split_rate_test SPLIT_RATE_TEST]
                             [--batch_size_train BATCH_SIZE_TRAIN]
                             [--batch_size_test BATCH_SIZE_TEST]
                             [--data_set DATA_SET] [--data_dir DATA_DIR]
                             [--checkpoint_dir CHECKPOINT_DIR]
                             [--result_dir RESULT_DIR] [--gpu_ratio GPU_RATIO]
                             [--cpu_num CPU_NUM] [--gpu_device GPU_DEVICE]
ipyker

SystemExit: 2

In [ ]:
predicted_class, target_class, words, model, input_embedded = test.predict(args, preprocessor)
print("Prediction: %s" % predicted_class)
target_class = predicted_class

if isinstance(target_class, str):
    target_class = int(target_class)


In [ ]:
# LRP hyperparameters:
eps = 0.001  # small positive number
bias_factor = 0.0  # recommended value
net = LSTM_bidi(args, model, input_embedded)  # load trained LSTM model

# w_indices = [net.voc.index(p) for p in prefix]  # convert input sentence to word IDs
Rx, Rx_rev, R_rest = net.lrp(words, target_class, eps, bias_factor)  # perform LRP
R_words = np.sum(Rx + Rx_rev, axis=1)  # compute word-level LRP relevances
scores = net.s.copy()  # classification prediction scores
print("prediction scores:", scores)
print("\nLRP target class:", target_class)
print("\nLRP relevances:")
for idx, w in enumerate(words):
    print("\t\t\t" + "{:8.10f}".format(R_words[idx]) + "\t" + w)
print("\nLRP heatmap:")
display(HTML(html_heatmap(words, R_words)))


In [ ]:
bias_factor = 1.0  # value to use for sanity check
Rx, Rx_rev, R_rest = net.lrp(words, target_class, eps, bias_factor)  # prefix -> w_indices
R_tot = Rx.sum() + Rx_rev.sum() + R_rest.sum()  # sum of all "input" relevances

print(R_tot)
print("Sanity check passed? ", np.allclose(R_tot, net.s[target_class]))